In [ ]:
%pip install pandas langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt langchain_upstage

In [ ]:
import os 
import dotenv 

dotenv.load_dotenv()

In [ ]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint

# GPT_API_KEY = os.environ.get("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [5]:
folder_path = '../data'

In [ ]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

In [10]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [11]:
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-index'

In [ ]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [13]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [ ]:
query = ('회수가 불필요한 상품')

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

# LLM 질의 테스트

In [ ]:
%pip install langchain_google_genai

In [ ]:
# ChatGoogleGenerativeAI 언어 모델을 초기화합니다.
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",  # 사용할 모델을 지정합니다.
)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답하세요.
단, [context] 정보에 없는 내용을 답해서는 안됩니다. 최대한 문장을 쉼표로 끊어서 대답하기 보다는, 온점으로 문장을 끊어주세요.
이 모든 정보를 종합해서 2~3줄의 구어체로 답해주세요.

"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke("상품의 상태를 확인").replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')
    